# Day 24: A2A-ify Anything - Agent2Agent 协议实战

## 学习目标

1. 理解 A2A 协议的核心概念
2. 学会创建 Agent Card
3. 实现 A2A Server 和 Client
4. 构建多 Agent 协作系统

## 1. A2A 协议基础

A2A (Agent2Agent) 是 Google 发布的开放协议，让不同框架构建的 Agent 能够互相通信。

In [ ]:
# 模拟 A2A 协议的核心数据结构
from dataclasses import dataclass, field
from typing import Optional, Any
from enum import Enum
import json
import uuid

class TaskStatus(Enum):
    """任务状态"""
    SUBMITTED = "submitted"
    WORKING = "working"
    INPUT_REQUIRED = "input_required"
    COMPLETED = "completed"
    FAILED = "failed"
    CANCELED = "canceled"

@dataclass
class TextPart:
    """文本消息部分"""
    text: str
    type: str = "text"

@dataclass
class DataPart:
    """结构化数据部分"""
    data: dict
    type: str = "data"

@dataclass
class Message:
    """A2A 消息"""
    role: str  # "user" or "agent"
    parts: list  # TextPart, DataPart, FilePart, etc.

@dataclass
class AgentSkill:
    """Agent 技能定义"""
    id: str
    name: str
    description: str
    input_schema: Optional[dict] = None

@dataclass
class AgentCard:
    """Agent 能力名片"""
    name: str
    description: str
    url: str
    version: str = "1.0.0"
    capabilities: dict = field(default_factory=lambda: {"streaming": False})
    skills: list = field(default_factory=list)
    
    def to_json(self) -> str:
        return json.dumps({
            "name": self.name,
            "description": self.description,
            "url": self.url,
            "version": self.version,
            "capabilities": self.capabilities,
            "skills": [
                {
                    "id": s.id,
                    "name": s.name,
                    "description": s.description,
                    "inputSchema": s.input_schema
                } for s in self.skills
            ]
        }, indent=2, ensure_ascii=False)

@dataclass
class TaskRequest:
    """任务请求"""
    message: Message
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    session_id: Optional[str] = None

@dataclass
class TaskResponse:
    """任务响应"""
    id: str
    status: TaskStatus
    messages: list = field(default_factory=list)
    artifacts: list = field(default_factory=list)

## 2. 创建 Agent Card

Agent Card 是 A2A 的核心概念，用于声明 Agent 的能力。

In [ ]:
# 创建一个报销处理 Agent 的 Agent Card
expense_agent_card = AgentCard(
    name="expense-reimbursement-agent",
    description="处理员工报销申请的 AI Agent，支持查询、提交和审批报销",
    url="http://localhost:8001",
    version="1.0.0",
    capabilities={
        "streaming": True,
        "pushNotifications": False,
        "stateTransitionHistory": True
    },
    skills=[
        AgentSkill(
            id="query-expenses",
            name="查询报销",
            description="查询员工的报销记录和状态",
            input_schema={
                "type": "object",
                "properties": {
                    "employee_id": {"type": "string", "description": "员工ID"},
                    "status": {"type": "string", "enum": ["pending", "approved", "rejected"]}
                },
                "required": ["employee_id"]
            }
        ),
        AgentSkill(
            id="submit-expense",
            name="提交报销",
            description="提交新的报销申请",
            input_schema={
                "type": "object",
                "properties": {
                    "amount": {"type": "number", "description": "报销金额"},
                    "category": {"type": "string", "description": "报销类别"},
                    "description": {"type": "string", "description": "报销说明"}
                },
                "required": ["amount", "category"]
            }
        ),
        AgentSkill(
            id="approve-expense",
            name="审批报销",
            description="审批待处理的报销申请",
            input_schema={
                "type": "object",
                "properties": {
                    "expense_id": {"type": "string", "description": "报销ID"},
                    "approved": {"type": "boolean", "description": "是否批准"}
                },
                "required": ["expense_id", "approved"]
            }
        )
    ]
)

print("📇 Agent Card (/.well-known/agent.json):")
print(expense_agent_card.to_json())

## 3. 模拟 A2A Server

实现一个简单的 A2A Server 来处理任务请求。

In [ ]:
from typing import Callable
import asyncio

class SimpleA2AServer:
    """简化的 A2A Server 实现"""
    
    def __init__(self, agent_card: AgentCard):
        self.agent_card = agent_card
        self.tasks: dict[str, TaskResponse] = {}
        self.handler: Optional[Callable] = None
    
    def set_handler(self, handler: Callable):
        """设置任务处理函数"""
        self.handler = handler
    
    def get_agent_card(self) -> str:
        """GET /.well-known/agent.json"""
        return self.agent_card.to_json()
    
    async def create_task(self, request: TaskRequest) -> TaskResponse:
        """POST /tasks - 创建新任务"""
        print(f"\n📥 收到任务请求: {request.id}")
        print(f"   消息: {request.message.parts[0].text}")
        
        # 创建初始响应
        response = TaskResponse(
            id=request.id,
            status=TaskStatus.WORKING,
            messages=[]
        )
        self.tasks[request.id] = response
        
        # 处理任务
        if self.handler:
            try:
                result = await self.handler(request)
                response.status = TaskStatus.COMPLETED
                response.messages.append(
                    Message(
                        role="agent",
                        parts=[TextPart(text=result)]
                    )
                )
            except Exception as e:
                response.status = TaskStatus.FAILED
                response.messages.append(
                    Message(
                        role="agent",
                        parts=[TextPart(text=f"错误: {str(e)}")]
                    )
                )
        
        print(f"   状态: {response.status.value}")
        return response
    
    def get_task(self, task_id: str) -> Optional[TaskResponse]:
        """GET /tasks/{id} - 获取任务状态"""
        return self.tasks.get(task_id)

In [ ]:
# 模拟报销数据库
expense_db = {
    "EMP001": [
        {"id": "EXP001", "amount": 500, "category": "差旅", "status": "pending"},
        {"id": "EXP002", "amount": 200, "category": "餐饮", "status": "approved"},
    ],
    "EMP002": [
        {"id": "EXP003", "amount": 1500, "category": "设备", "status": "pending"},
    ]
}

async def expense_handler(request: TaskRequest) -> str:
    """报销 Agent 的请求处理函数"""
    message = request.message.parts[0].text.lower()
    
    # 简单的意图识别
    if "查询" in message or "报销" in message:
        # 查询报销
        employee_id = "EMP001"  # 简化：假设是 EMP001
        expenses = expense_db.get(employee_id, [])
        
        result = f"员工 {employee_id} 的报销记录:\n"
        for exp in expenses:
            result += f"  - {exp['id']}: ¥{exp['amount']} ({exp['category']}) - {exp['status']}\n"
        return result
    
    elif "批准" in message or "审批" in message:
        # 审批报销
        return "已批准报销申请 EXP001，金额 ¥500"
    
    elif "提交" in message:
        # 提交新报销
        new_id = f"EXP{len(expense_db.get('EMP001', [])) + 100}"
        return f"已创建新的报销申请 {new_id}，等待审批"
    
    else:
        return "我是报销处理 Agent，可以帮您查询、提交或审批报销申请。请问有什么需要帮助的？"

# 创建并配置 Server
expense_server = SimpleA2AServer(expense_agent_card)
expense_server.set_handler(expense_handler)

print("✅ 报销 Agent Server 已创建")

## 4. 模拟 A2A Client

实现一个 A2A Client 来与 Server 通信。

In [ ]:
class SimpleA2AClient:
    """简化的 A2A Client 实现"""
    
    def __init__(self, server: SimpleA2AServer):
        # 在真实实现中，这里是 HTTP URL
        self.server = server
        self._agent_card: Optional[AgentCard] = None
    
    def discover(self) -> AgentCard:
        """发现远程 Agent 的能力"""
        card_json = self.server.get_agent_card()
        card_data = json.loads(card_json)
        
        self._agent_card = AgentCard(
            name=card_data["name"],
            description=card_data["description"],
            url=card_data["url"],
            version=card_data["version"],
            capabilities=card_data["capabilities"],
            skills=[
                AgentSkill(
                    id=s["id"],
                    name=s["name"],
                    description=s["description"],
                    input_schema=s.get("inputSchema")
                ) for s in card_data["skills"]
            ]
        )
        return self._agent_card
    
    async def send_task(self, message: str) -> TaskResponse:
        """发送任务到远程 Agent"""
        request = TaskRequest(
            message=Message(
                role="user",
                parts=[TextPart(text=message)]
            )
        )
        return await self.server.create_task(request)
    
    def get_skills(self) -> list[AgentSkill]:
        """获取 Agent 的技能列表"""
        if self._agent_card is None:
            self.discover()
        return self._agent_card.skills

In [ ]:
# 创建 Client 并测试
client = SimpleA2AClient(expense_server)

# 1. 发现 Agent 能力
print("="*60)
print("Step 1: 发现 Agent 能力")
print("="*60)

agent_card = client.discover()
print(f"\n🤖 Agent: {agent_card.name}")
print(f"📝 描述: {agent_card.description}")
print(f"🔗 URL: {agent_card.url}")
print(f"\n📋 技能列表:")
for skill in agent_card.skills:
    print(f"   - {skill.name}: {skill.description}")

In [ ]:
# 2. 发送任务
print("\n" + "="*60)
print("Step 2: 发送任务 - 查询报销")
print("="*60)

response = await client.send_task("查询我的报销记录")
print(f"\n📤 任务ID: {response.id}")
print(f"📊 状态: {response.status.value}")
print(f"\n💬 Agent 回复:")
for msg in response.messages:
    print(msg.parts[0].text)

In [ ]:
# 3. 发送另一个任务 - 审批
print("\n" + "="*60)
print("Step 3: 发送任务 - 审批报销")
print("="*60)

response = await client.send_task("批准 EXP001 的报销申请")
print(f"\n📤 任务ID: {response.id}")
print(f"📊 状态: {response.status.value}")
print(f"\n💬 Agent 回复:")
for msg in response.messages:
    print(msg.parts[0].text)

## 5. 多 Agent 协作

演示多个 A2A Agent 如何协作完成复杂任务。

In [ ]:
# 创建采购目录 Agent
catalog_agent_card = AgentCard(
    name="catalog-agent",
    description="产品目录查询 Agent，提供产品搜索和价格信息",
    url="http://localhost:8002",
    skills=[
        AgentSkill(
            id="search-products",
            name="搜索产品",
            description="根据关键词搜索产品"
        )
    ]
)

# 产品数据库
product_db = [
    {"id": "PRD001", "name": "MacBook Pro 14", "price": 15999, "category": "电脑"},
    {"id": "PRD002", "name": "Dell XPS 15", "price": 12999, "category": "电脑"},
    {"id": "PRD003", "name": "ThinkPad X1 Carbon", "price": 11999, "category": "电脑"},
]

async def catalog_handler(request: TaskRequest) -> str:
    message = request.message.parts[0].text.lower()
    
    results = []
    for product in product_db:
        if any(keyword in product["name"].lower() for keyword in message.split()):
            results.append(product)
    
    if not results:
        results = product_db  # 返回所有产品
    
    result = "找到以下产品:\n"
    for p in results:
        result += f"  - {p['name']}: ¥{p['price']}\n"
    return result

catalog_server = SimpleA2AServer(catalog_agent_card)
catalog_server.set_handler(catalog_handler)

In [ ]:
# 创建预算检查 Agent
budget_agent_card = AgentCard(
    name="budget-agent",
    description="预算管理 Agent，检查部门预算和采购权限",
    url="http://localhost:8003",
    skills=[
        AgentSkill(
            id="check-budget",
            name="检查预算",
            description="检查指定金额是否在预算范围内"
        )
    ]
)

# 部门预算
department_budgets = {
    "engineering": 50000,
    "marketing": 30000,
    "sales": 40000
}

async def budget_handler(request: TaskRequest) -> str:
    message = request.message.parts[0].text
    
    # 简单解析金额
    import re
    amounts = re.findall(r'\d+', message)
    if amounts:
        amount = int(amounts[0])
        budget = department_budgets.get("engineering", 0)
        
        if amount <= budget:
            return f"✅ 预算检查通过！申请金额 ¥{amount} 在部门预算 ¥{budget} 范围内。"
        else:
            return f"❌ 预算不足！申请金额 ¥{amount} 超出部门预算 ¥{budget}。"
    
    return "请提供具体的采购金额。"

budget_server = SimpleA2AServer(budget_agent_card)
budget_server.set_handler(budget_handler)

In [ ]:
# 创建协调者 Agent
class ConciergeAgent:
    """采购协调者 Agent - 协调多个 A2A Agent 完成采购流程"""
    
    def __init__(self):
        self.catalog_client = SimpleA2AClient(catalog_server)
        self.budget_client = SimpleA2AClient(budget_server)
        self.expense_client = SimpleA2AClient(expense_server)
    
    async def process_purchase_request(self, request: str) -> str:
        """处理采购请求 - 协调多个 Agent"""
        print("\n" + "="*60)
        print("🎯 采购协调者 Agent 开始处理")
        print(f"📝 用户请求: {request}")
        print("="*60)
        
        results = []
        
        # Step 1: 搜索产品目录
        print("\n📦 Step 1: 调用 Catalog Agent 搜索产品...")
        catalog_response = await self.catalog_client.send_task(request)
        catalog_result = catalog_response.messages[0].parts[0].text
        print(f"\n{catalog_result}")
        results.append(catalog_result)
        
        # 从结果中提取价格（简化处理）
        import re
        prices = re.findall(r'¥(\d+)', catalog_result)
        if prices:
            selected_price = prices[0]  # 选择第一个产品
            
            # Step 2: 检查预算
            print(f"\n💰 Step 2: 调用 Budget Agent 检查预算 (¥{selected_price})...")
            budget_response = await self.budget_client.send_task(f"检查预算 {selected_price}")
            budget_result = budget_response.messages[0].parts[0].text
            print(f"\n{budget_result}")
            results.append(budget_result)
            
            # Step 3: 如果预算通过，创建报销申请
            if "通过" in budget_result:
                print(f"\n📋 Step 3: 调用 Expense Agent 提交采购申请...")
                expense_response = await self.expense_client.send_task(f"提交采购申请 ¥{selected_price}")
                expense_result = expense_response.messages[0].parts[0].text
                print(f"\n{expense_result}")
                results.append(expense_result)
        
        # 汇总结果
        print("\n" + "="*60)
        print("📊 处理完成 - 汇总")
        print("="*60)
        
        return "\n\n".join(results)

In [ ]:
# 测试多 Agent 协作
concierge = ConciergeAgent()

result = await concierge.process_purchase_request("我需要购买一台 MacBook 用于开发工作")

print("\n" + "="*60)
print("✅ 最终结果")
print("="*60)
print(result)

## 6. Agent Starter Pack 使用指南

在实际项目中，使用 Agent Starter Pack 可以快速创建 A2A Agent。

In [ ]:
starter_pack_guide = """
# Agent Starter Pack 快速开始

## 1. 创建新的 A2A Agent

```bash
# 使用 uvx（无需安装）
uvx agent-starter-pack create my-agent \
    -a adk@deep-search \
    --base-template adk_a2a_base

# 进入项目
cd my-agent

# 设置环境变量
export GOOGLE_API_KEY="your-api-key"

# 启动 Agent
uv run .
```

## 2. 为现有 Agent 添加 A2A 能力

```bash
# 进入现有项目目录
cd existing-agent

# 增强 Agent
uvx agent-starter-pack enhance --base-template adk_a2a_base
```

## 3. 可用模板

| 模板 | 说明 |
|------|------|
| `adk@deep-search` | 深度搜索 Agent（网络搜索） |
| `adk@rag` | RAG Agent（检索增强生成） |
| `adk@multi-agent` | 多 Agent 协作系统 |
| `adk@live` | 实时交互 Agent |

## 4. 项目结构

```
my-agent/
├── pyproject.toml        # 依赖配置
├── __main__.py           # 入口文件
├── agent/
│   ├── __init__.py
│   ├── agent.py          # Agent 定义
│   ├── tools.py          # 工具定义
│   └── prompts.py        # Prompt 模板
├── a2a/
│   ├── server.py         # A2A Server
│   └── agent_card.py     # Agent Card
└── tests/
    └── test_agent.py
```

## 5. 测试 A2A Agent

```bash
# 查看 Agent Card
curl http://localhost:8000/.well-known/agent.json

# 发送任务
curl -X POST http://localhost:8000/tasks \
  -H "Content-Type: application/json" \
  -d '{
    "message": {
      "role": "user",
      "parts": [{"text": "你好，请帮我搜索..."}]
    }
  }'
```
"""

print(starter_pack_guide)

## 7. 总结

In [ ]:
summary = """
╔══════════════════════════════════════════════════════════════════╗
║              Day 24 关键概念总结                                   ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  1. A2A 协议核心概念                                              ║
║     • Agent Card: 声明 Agent 能力的 JSON 描述                     ║
║     • Task: 有状态的任务，支持长时间运行                           ║
║     • Message Parts: 多模态消息内容                               ║
║                                                                  ║
║  2. A2A vs MCP                                                   ║
║     • A2A: Agent 之间的水平通信                                   ║
║     • MCP: Agent 到工具的垂直访问                                 ║
║     • 两者互补，可同时使用                                        ║
║                                                                  ║
║  3. Agent Starter Pack                                           ║
║     • 快速创建: uvx agent-starter-pack create                    ║
║     • 增强现有: uvx agent-starter-pack enhance                   ║
║     • 多种模板: deep-search, rag, multi-agent, live              ║
║                                                                  ║
║  4. 多 Agent 协作模式                                             ║
║     • Concierge Agent: 协调者模式                                ║
║     • 能力发现: 通过 Agent Card 发现远程 Agent                    ║
║     • 任务编排: 串行/并行调用多个 Agent                           ║
║                                                                  ║
║  5. 关键端点                                                      ║
║     • /.well-known/agent.json: Agent Card                        ║
║     • /tasks: 创建任务                                           ║
║     • /tasks/{id}: 查询任务状态                                   ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
"""

print(summary)

## 参考资源

- [A2A Protocol 官方文档](https://a2a-protocol.org/)
- [A2A GitHub 仓库](https://github.com/a2aproject/A2A)
- [Agent Starter Pack](https://github.com/GoogleCloudPlatform/agent-starter-pack)
- [Google ADK Samples](https://github.com/google/adk-samples)
- [A2A Codelab](https://codelabs.developers.google.com/intro-a2a-purchasing-concierge)
- [A2A 发布公告](https://developers.googleblog.com/en/a2a-a-new-era-of-agent-interoperability/)